In [1]:
import redcap
import pandas as pd
import datetime
data = {
    'token': 'E5D1F8B694072DCC46C0A8236CCC2670',
    'content': 'project',
    'format': 'json',
    'returnFormat': 'json'
}
Sen_Data = redcap.Project('https://redcap.mrc.gm:8443/redcap/api/', data['token'])
# IDeA connection

data = {
    'token': '085520489DF98911614E03FF05D58EA8',
    'content': 'project',
    'format': 'csv',
    'returnFormat': 'csv'
}

IdeA_Data = redcap.Project('https://ruff.mrc.gm:8443/redcap/api/', data['token'])

In [8]:
#consent data from IDeA 3 study
consent_Data1=IdeA_Data.export_report(report_id='264')
consent_Data1=pd.DataFrame(consent_Data1)
consent_Data1=consent_Data1[['cons_recordid_q1', 'cons_visitdate_q2',
       'cons_obtained_q4', 'cons_provided_infconsent_q7', 'cons_lang_cobtained_q10','cons_scompletedid_q13']]
consent_Data1 = consent_Data1[consent_Data1['cons_recordid_q1'].str.startswith('IDeA')]
consent_Data1=consent_Data1.rename(columns={'cons_recordid_q1': 'record_id'})
consent_Data1=consent_Data1.rename(columns={'cons_visitdate_q2': 'Date of Consenting'})
consent_Data1=consent_Data1.rename(columns={'cons_obtained_q4': 'Was consent obtained'})
consent_Data1=consent_Data1.rename(columns={'cons_provided_infconsent_q7': 'Who gave consent'})
consent_Data1=consent_Data1.rename(columns={'cons_lang_cobtained_q10': 'Language of consent'})
consent_Data1=consent_Data1.rename(columns={'cons_scompletedid_q13': 'Staff ID'})

consent_Data1=consent_Data1[
       (consent_Data1['Was consent obtained']=='1')
]


#sensitization data from the sensitization

Sensitization_data1=Sen_Data.export_records(forms=['sensitization'])
Sensitization_data1=pd.DataFrame(Sensitization_data1)
Sensitization_data1=Sensitization_data1[['record_id','part_name_ff615e',]]
Sensitization_data1=Sensitization_data1.rename(columns={'part_name_ff615e': 'Name of Participant'})
Sensitization_data1['record_id'] = Sensitization_data1['record_id'].str.replace('SEN', '')

#merge data between the consent and the sensitization data
PROPOSED_CONSENT_LOG=pd.merge(Sensitization_data1,consent_Data1,on='record_id', how='inner')
PROPOSED_CONSENT_LOG.to_csv('PROPOSED_CONSENT_LOG.csv',index=False)


In [9]:
filter_consent_no=consent_Data1[
    consent_Data1['Was consent obtained']=='1']

In [10]:
Screening_Data1=IdeA_Data.export_report(report_id='265')
Screening_Data1=pd.DataFrame(Screening_Data1)
Screening_Data1 = Screening_Data1[Screening_Data1['cons_recordid_q1'].str.startswith('IDeA')]
Screening_Data1=Screening_Data1[['cons_recordid_q1','scre_date_q1','scre_compstaffid_q15']]
Screening_Data1=Screening_Data1.rename(columns={'cons_recordid_q1': 'record_id'})
Screening_Data1=Screening_Data1.rename(columns={'scre_date_q1': 'Date Screened'})
Screening_Data1=Screening_Data1.rename(columns={'scre_compstaffid_q15': 'Staff ID'})

Screening_Data1.to_csv('Screening_Data1.csv',index=False)



In [11]:
Randomization=IdeA_Data.export_report(report_id='266')
Randomization=pd.DataFrame(Randomization)
Randomization=Randomization[['cons_recordid_q1','rand_date_q1','rand_treatgroup_q3']]
Randomization= Randomization[Randomization['cons_recordid_q1'].str.startswith('IDeA')]
Randomization['Saff']='MB'
Randomization=Randomization.rename(columns={'cons_recordid_q1': 'record_id'})
Randomization=Randomization.rename(columns={'rand_date_q1': 'Date of Randomization'})
Randomization=Randomization.rename(columns={'rand_treatgroup_q3': 'Treatment Group'})
Randomization.to_csv('Randomization.csv',index=False)



In [12]:
withdraw=IdeA_Data.export_records(forms=['withdrawal'])
withdraw=pd.DataFrame(withdraw)

withdraw=withdraw[
    (withdraw['withdrawal_complete']=='2')
]

In [13]:

resent_Randomization=IdeA_Data.export_report(report_id='266')
resent_Randomization=pd.DataFrame(resent_Randomization)
just_Randomization= resent_Randomization[~resent_Randomization['cons_recordid_q1'].isin(withdraw['cons_recordid_q1'])]

In [14]:
Screen_Ran=pd.merge(Screening_Data1,Randomization,on='record_id',how='inner')
screen_Ran_sen=pd.merge(Sensitization_data1,Screen_Ran,on='record_id',how='right')
screen_Ran_sen.to_csv('screen_Ran_sen.csv',index=False)


In [15]:
# fail screening 
fail_screen=IdeA_Data.export_report(report_id='267')
fail_screen=pd.DataFrame(fail_screen)
fail_screen=fail_screen.rename(columns={'cons_recordid_q1': 'record_id'})
fail_screen = fail_screen[fail_screen['record_id'].str.startswith('IDeA')]
fail_screen=fail_screen[['record_id']]
fail_screen['Participant Eligible']='no'
fail_screen['If Not Eligible state reason']='Have HB <7.0 or >11.0'
fail_screen.to_csv('fail_screen.csv',index=False)

In [16]:
Fail_Pre_Screen=IdeA_Data.export_report(report_id='269')
Fail_Pre_Screen=pd.DataFrame(Fail_Pre_Screen)
Fail_Pre_Screen=Fail_Pre_Screen.rename(columns={'cons_recordid_q1': 'record_id'})
Fail_Pre_Screen=Fail_Pre_Screen[['record_id','pres_vdate_q1']]
Fail_Pre_Screen=Fail_Pre_Screen.rename(columns={'pres_vdate_q1': 'Prescreening Date'})
Fail_Pre_Screen['Participant Eligible']='no'
Fail_Pre_Screen['If Not Eligible state reason']='Have HB <7.0 or >11.0'
Fail_Pre_Screen= Fail_Pre_Screen[Fail_Pre_Screen['record_id'].str.startswith('IDeA')]
Fail_Pre_screen_with_name=pd.merge(Sensitization_data1,Fail_Pre_Screen,on='record_id',how='inner')
Fail_Pre_Screen.to_csv('Fail_Pre_Screen.csv',index=False)

In [17]:
just_Fail_Pre_Screen=IdeA_Data.export_report(report_id='269')
just_Fail_Pre_Screen=pd.DataFrame(just_Fail_Pre_Screen)
just_Fail_Pre_Screen= just_Fail_Pre_Screen[~just_Fail_Pre_Screen['cons_recordid_q1'].isin(withdraw['cons_recordid_q1'])]

In [18]:
pre_screen_data=IdeA_Data.export_records(forms=['prescreening'])
pre_screen_data=pd.DataFrame(pre_screen_data)
pre_screen_data= pre_screen_data[pre_screen_data['cons_recordid_q1'].str.startswith('IDeA')]
pre_screen_data=pre_screen_data[
    pre_screen_data['prescreening_complete']=='2'
]

just_pre_pre_screen_data= pre_screen_data[~pre_screen_data['cons_recordid_q1'].isin(withdraw['cons_recordid_q1'])]


In [19]:
pre_screen_participant_seen=pre_screen_data[
    (pre_screen_data['a_was_participant_seen']=='1')
]
just_pre_screen_participant_seen= pre_screen_participant_seen[~pre_screen_participant_seen['cons_recordid_q1'].isin(withdraw['cons_recordid_q1'])]

In [20]:
could_not_be_located=pre_screen_data[
    pre_screen_data['q1b_if_no_give_reason']=='1'
]
just_could_not_be_located= could_not_be_located[~could_not_be_located['cons_recordid_q1'].isin(withdraw['cons_recordid_q1'])]

In [21]:
Pre_screen_Travel=pre_screen_data[
    pre_screen_data['q1b_if_no_give_reason']=='2'
]

just_Pre_screen_Travel= Pre_screen_Travel[~Pre_screen_Travel['cons_recordid_q1'].isin(withdraw['cons_recordid_q1'])]

In [22]:
Pre_Screen_Decline=pre_screen_data[
    pre_screen_data['q2a_if_no_give_reason']=='1'
]

just_decline= Pre_Screen_Decline[~Pre_Screen_Decline['cons_recordid_q1'].isin(withdraw['cons_recordid_q1'])]

In [23]:
#Master Screening list Both eligible and ineligible 
screening_list_completed=IdeA_Data.export_report(report_id='268')
screening_list_completed=pd.DataFrame(screening_list_completed)
screening_list_completed=screening_list_completed.rename(columns={'cons_recordid_q1': 'record_id'})
screening_list_completed= screening_list_completed[screening_list_completed['record_id'].str.startswith('IDeA')]
screening_list_completed=screening_list_completed[['record_id','scre_date_q1','scre_compstaffid_q15']]


In [24]:
pre_screened_data_oko=IdeA_Data.export_records(forms=['prescreening'])
pre_screened_data_oko=pd.DataFrame(pre_screened_data_oko)
pre_screened_data_oko= pre_screened_data_oko[pre_screened_data_oko['cons_recordid_q1'].str.startswith('IDeA')]
pre_screened_data_oko=pre_screened_data_oko[pre_screened_data_oko
                                    ['prescreening_complete']=='2']
just_pre_screened_data_oko = pre_screened_data_oko[~pre_screened_data_oko['cons_recordid_q1'].isin(withdraw['cons_recordid_q1'])]

# Assuming you have already defined pre_screened_data_oko DataFrame

# Convert 'pres_hemoglobin_q2' column to numeric data type
pre_screened_data_oko['pres_hemoglobin_q2'] = pd.to_numeric(pre_screened_data_oko['pres_hemoglobin_q2'], errors='coerce')

# Filter rows with 'pres_hemoglobin_q2' values between 7.0 and 11.0
Pre_screen_filtered_data_pass_HB = pre_screened_data_oko [
    (pre_screened_data_oko['pres_hemoglobin_q2'] < 11.0) 
]

fail2= pre_screened_data_oko [ 
    (pre_screened_data_oko['pres_hemoglobin_q2']< 7.0)
]
# Now 'filtered_data' contains the rows that meet the condition
just_Pre_screen_filtered_data_pass_HB= Pre_screen_filtered_data_pass_HB[~Pre_screen_filtered_data_pass_HB['cons_recordid_q1'].isin(withdraw['cons_recordid_q1'])]

In [25]:
blood_sample_collected_okoh=IdeA_Data.export_records(forms=['bloodsamplecollection'])
blood_sample_collected_okoh=pd.DataFrame(blood_sample_collected_okoh)
blood_sample_collected_okoh= blood_sample_collected_okoh[blood_sample_collected_okoh['cons_recordid_q1'].str.startswith('IDeA')]
blood_sample_collected_okoh=blood_sample_collected_okoh[blood_sample_collected_okoh
                                    ['bloodsamplecollection_complete']=='2']


In [26]:
import pandas as pd

# Assuming you have already defined pre_screened_data_oko and blood_sample_collected_okoh DataFrames

# Merge the two DataFrames using 'cons_recordid_q1' as the key and indicator=True
merged_data = Pre_screen_filtered_data_pass_HB.merge(blood_sample_collected_okoh, on='cons_recordid_q1', how='left', indicator=True)

# Filter rows that exist in pre_screened_data_oko but not in blood_sample_collected_okoh
not_collected_data = merged_data[merged_data['_merge'] == 'left_only']

not_collected_data.to_csv('not_collected_data.csv',index=False)
# Now 'not_collected_data' contains individuals who are in pre_screened_data_oko but not in blood_sample_collected_okoh


to be completed later

screening_blood_collection_Miss_Match_okoh= (thoes with  HB - fail_Pre_screen) - (blood_sample_completed) about 17 IDs 

In [27]:
prescreening=IdeA_Data.export_report(report_id='334')
prescree_with_HB=pd.DataFrame(prescreening)
just_prescree_with_HB= prescree_with_HB[~prescree_with_HB['cons_recordid_q1'].isin(withdraw['cons_recordid_q1'])]

In [2]:
IdeA_Data.forms

['sourcedocument',
 'consent',
 'noncompliance_form',
 'demograhics',
 'sysmexresult',
 'serumstorage',
 'samplereception',
 'ironpanel1',
 'vitalsign',
 'dailyhealth',
 'screening',
 'daily_diarrhoea_questionaire',
 'assessment_of_informed_consent_understanding',
 'stoolstorage',
 'medonicresult',
 'withdrawal',
 'hepcidineresults',
 'intervention',
 'clinicreferal',
 'stool_processing',
 'randomization',
 'ironsupplement',
 'bloodsample_processing',
 'g6pd',
 'ip_accountability_log',
 'ironsupplement_unplanned',
 'ironpanel1_2',
 'anthrops',
 'feedingquestionaire',
 'appetite',
 'stoolsamplecollection',
 'calprotectin',
 'prescreening',
 'ae',
 'erythroferroneresult',
 'posthealthmonitoring',
 'bloodsamplecollection',
 'g6pd_819c16',
 'erythroferroneresult_2']

In [4]:
iron_supplementation=IdeA_Data.export_records(forms=['ironsupplement','sysmexresult'])

In [5]:
iron_supplementation=pd.DataFrame(iron_supplementation)

In [ ]:
iron_supplementation=iron_supplementation[
    (iron_supplementation['sysmexresult_complete']=='2')&
    (iron_supplementation['redcap_event_name']=='v12_arm_1')
]